# Toffoli

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran import QBit, QInt, QUInt, QAny
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `Toffoli`
The Toffoli gate.

This will flip the target bit if both controls are active. It can be thought of as
a reversible AND gate.

Like `TGate`, this is a common compilation target. The Clifford+Toffoli gateset is
universal.

#### References
 - [Novel constructions for the fault-tolerant Toffoli gate](https://arxiv.org/abs/1212.5069). Cody Jones. 2012. Provides a decomposition into 4 `TGate`.


In [ ]:
from qualtran.bloqs.basic_gates import Toffoli

### Example Instances

In [ ]:
toffoli = Toffoli()

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([toffoli],
           ['`toffoli`'])

In [ ]:
show_bloq(toffoli, 'musical_score')

### Call Graph

In [ ]:
from qualtran.resource_counting.generalizers import ignore_split_join
toffoli_g, toffoli_sigma = toffoli.call_graph(max_depth=1, generalizer=ignore_split_join)
show_call_graph(toffoli_g)
show_counts_sigma(toffoli_sigma)

## Controlled

We use `Toffoli` for specifically two control bits of the `XGate`. Requesting `Toffoli().controlled(ctrl_spec)` will give an instance of the generic `ControlledViaAnd(XGate(), ...)` bloq, which will control the `XGate` on an arbitrary boolean function supported by `CtrlSpec`. The set of control registers present on the resulting bloq will 1) the control registers needed for the provided `ctrl_spec` and 2) one additional `QBit()[2]` register for the original Toffoli control bits.

In [ ]:
from qualtran import CtrlSpec
print(repr(Toffoli().controlled()))
show_bloq(Toffoli().controlled(), 'musical_score')

In [ ]:
print(repr(Toffoli().controlled(CtrlSpec(cvs=0))))
show_bloq(Toffoli().controlled(CtrlSpec(cvs=0)), 'musical_score')

### Complex control specs

Per above, `Toffoli.controlled()` can support arbitrary control specs.

In [ ]:
active_six = CtrlSpec(qdtypes=QInt(8), cvs=6)
six_and_toffoli = Toffoli().controlled(active_six)
print(repr(six_and_toffoli))
show_bloq(six_and_toffoli, 'musical_score')

### Calling controlled gates automatically

The `Toffoli().controlled()` method (and in general, the `.controlled()` method for any bloq object) returns a bloq that implements a controlled version of the original gate. The returned bloq can structure its signature however it chooses. You can see that `Toffoli.controlled()` returns a `ControlledViaAnd(XGate(), ...)` which names its control registers `ctrl{n}` with the final one corresponding to the original Toffoli control bits. 

When writing meta-bloqs that need to support adding arbitrarily-controlled versions of Toffoli to a decomposition, you should not rely on any particular signature. `Bloq.get_ctrl_system()` is the method for automatically `bb.add`-ing controlled versions of arbitrary bloqs. We demonstrate that below.

In [ ]:
from qualtran.bloqs.basic_gates import IntState, IntEffect

bb = BloqBuilder()
# q0, q1, target will be original wires for calling Toffoli
q0 = bb.add_register('q0', QBit())
q1 = bb.add_register('q1', QBit())
target = bb.add_register('target', QBit())

# "six" is a new wire that we additionally want to control the Toffoli with
six = bb.add(IntState(val=6, bitsize=8))
ctrl_spec = CtrlSpec(qdtypes=QUInt(8), cvs=6)

# Instead of using Toffoli().controlled() directly, we use the ctrl_system
ctof, ctof_adder = Toffoli().get_ctrl_system(ctrl_spec)

# The new control variables are passed in their own list and returned in their own list
# The existing Toffoli inputs are provided in a dictionary and returned in their own list
(six,), ((q0, q1), target) = ctof_adder(bb, ctrl_soqs=[six], in_soqs=dict(ctrl=[q0,q1], target=target))

# # below is a brittle way to achieve the same effect for this particular ctrl_spec
# # it assumes control registers named ctrl1 and ctrl2
# six_and_toffoli = Toffoli().controlled(ctrl_spec)
# six, (q0, q1), target = bb.add(six_and_toffoli, ctrl1=six, ctrl2=[q0, q1], q=target)

bb.add(IntEffect(val=6, bitsize=8), val=six)
program = bb.finalize(q0=q0, q1=q1, target=target)
show_bloq(program, 'musical_score')